In [1]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from collections import defaultdict
import sys

In [3]:
distdeck = torch.load(
    "results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_1-distdeck.pth"
)

In [4]:
deck2 = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_1-deck2.pth")

In [5]:
df = pd.read_csv("data/ori/test.csv")

In [7]:
distdeck["dist"].shape, distdeck["tlevel"].shape

(torch.Size([1095951, 4]), torch.Size([1095951, 4]))

In [8]:
distdeck.keys(), deck2.keys()

(dict_keys(['dist', 'tlevel', 'fclevel', 'index', 'ids']),
 dict_keys(['feat', 'otext', 'fclevel', 'ids']))

In [9]:
total_len = 1418916

In [65]:
def policy(dists, tlevels, fclevel):
    if (tlevels[:3] == 5).all():
        return 5 if dists[0] < 1.2 else 7
    if (tlevels[:3] == 3).all():
        return 3 if dists[0] < 1.2 else 7
    if fclevel in [6, 4, 2]:
        return fclevel.item()
    if dists[0] < 0.7:
        # return tlevels[0].item()
        return fclevel.item()
    return 7

In [66]:
out_dists = [None for _ in range(total_len)]
out_levels = [None for _ in range(total_len)]
out_fclevels = [None for _ in range(total_len)]
N = distdeck["dist"].size(0)
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dists = distdeck["dist"][i]
        levels = distdeck["tlevel"][i]
        fclevel = deck2["fclevel"][i]
        out_level = policy(dists, levels, fclevel)
        ids = deck2["ids"][i]
        for j in ids:
            out_levels[j - 1000000] = out_level
            out_dists[j - 1000000] = dists
            out_fclevels[j - 1000000] = fclevel
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [01:43<00:00, 10624.89it/s]


In [67]:
out_levels = np.array(out_levels)

In [68]:
# 각 레벨별 개수 0.5이하일 때 fclevel을 쓰기로
for i in range(8):
    cnt = (out_levels == i).sum()
    print(i, ":", cnt, f"{cnt / len(out_levels)*100:.2f}%")

0 : 1001305 70.57%
1 : 394492 27.80%
2 : 48 0.00%
3 : 12831 0.90%
4 : 34 0.00%
5 : 6376 0.45%
6 : 31 0.00%
7 : 3799 0.27%


```
0 : 1003955 70.76%
1 : 395007 27.84%
2 : 42 0.00%
3 : 12950 0.91%
4 : 34 0.00%
5 : 6334 0.45%
6 : 31 0.00%
7 : 563 0.04%
```

In [39]:
(out_levels == 6).nonzero()

(array([  35574,   64268,   73308,   86431,   92457,  147553,  175492,
         179014,  210545,  240872,  340297,  355854,  374455,  451522,
         484782,  499683,  507877,  529914,  563330,  624698,  633202,
         647782,  652696,  657244,  667019,  714497,  848132,  881660,
        1113383, 1125342, 1169189, 1293703, 1394234]),)

In [15]:
xdf = pd.read_csv("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-out_ver2.csv")

In [16]:
xlevels = xdf.level.to_numpy()

In [17]:
(xlevels == 6).nonzero()

(array([  35574,   64268,   73308,   86431,   92457,  147553,  175492,
         179014,  210545,  240872,  340297,  355854,  374455,  451522,
         484782,  499683,  507877,  529914,  563330,  624698,  633202,
         647782,  652696,  714497,  848132,  881660, 1113383, 1125342,
        1169189, 1293703, 1394234]),)

In [18]:
tdf = pd.read_csv("data/ori/train.csv")

In [19]:
otexts = tdf.full_log.to_numpy()

In [20]:
def find(text):
    out_indices = []
    for i, otext in enumerate(otexts):
        if text in otext:
            return i


def find_all(text):
    out_indices = []
    for i, otext in enumerate(otexts):
        if text in otext:
            out_indices.append(i)
    return out_indices

In [63]:
find_all("ossec-control status")[:10]

[262, 362, 581, 1863, 2454, 3043, 4013, 4862, 5252, 6793]

In [64]:
tdf.level[262], tdf.full_log[262]

(1,
 'Mar  2 22:00:01 localhost sudo:    root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/var/esild/bin/ossec-control status')

In [69]:
for i in (xlevels == 2).nonzero()[0]:
    print(f"{i:8} | {str(df.full_log[i])}")

   13058 | The average number of logs between 6:00 and 7:00 is 6708. We reached 48588.
   19064 | The average number of logs between 18:00 and 19:00 is 5572. We reached 94025.
   50292 | The average number of logs between 20:00 and 21:00 is 4584. We reached 65427.
   77934 | The average number of logs between 7:00 and 8:00 is 2359. We reached 67869.
  136006 | The average number of logs between 3:00 and 4:00 is 2163. We reached 80857.
  236519 | The average number of logs between 11:00 and 12:00 is 9137. We reached 41070.
  247477 | The average number of logs between 8:00 and 9:00 is 8389. We reached 23698.
  274200 | The average number of logs between 1:00 and 2:00 is 4017. We reached 41065.
  286782 | The average number of logs between 0:00 and 1:00 is 12403. We reached 31009.
  304176 | The average number of logs between 0:00 and 1:00 is 6204. We reached 15512.
  333032 | The average number of logs between 0:00 and 1:00 is 2949. We reached 71943.
  344080 | Dec 15 14:23:41 localhost

In [70]:
for i in (out_levels == 2).nonzero()[0]:
    print(f"{i:8} | {str(df.full_log[i])}")

   13058 | The average number of logs between 6:00 and 7:00 is 6708. We reached 48588.
   19064 | The average number of logs between 18:00 and 19:00 is 5572. We reached 94025.
   31449 | Oct 14 15:10:00 localhost sudo:  apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/ls /var/esild/ruleset/rules/
   50292 | The average number of logs between 20:00 and 21:00 is 4584. We reached 65427.
   77934 | The average number of logs between 7:00 and 8:00 is 2359. We reached 67869.
  136006 | The average number of logs between 3:00 and 4:00 is 2163. We reached 80857.
  236519 | The average number of logs between 11:00 and 12:00 is 9137. We reached 41070.
  247477 | The average number of logs between 8:00 and 9:00 is 8389. We reached 23698.
  253931 | Oct 15 10:16:26 localhost sudo:  apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/cat /var/esild/ruleset/decoders/0120-horde_decoders.xml
  274200 | The average number of logs between 1:00 and

In [58]:
for i, (level, otext) in enumerate(zip(tdf.level, tdf.full_log)):
    if level == 2:
        print(f"{i:8} | {otext}")

   39057 | The average number of logs between 10:00 and 11:00 is 5399. We reached 37090.
   87600 | The average number of logs between 5:00 and 6:00 is 222. We reached 77946.
  109510 | The average number of logs between 6:00 and 7:00 is 6420. We reached 28494.
  262892 | The average number of logs between 21:00 and 22:00 is 5325. We reached 95.
  294072 | The average number of logs between 5:00 and 6:00 is 8362. We reached 93476.
  294127 | The average number of logs between 4:00 and 5:00 is 2806. We reached 11476.
  343011 | The average number of logs between 14:00 and 15:00 is 9734. We reached 26958.
  345903 | The average number of logs between 15:00 and 16:00 is 4411. We reached 9487.
  365348 | Sep 18 11:02:59 localhost sudo:  apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/curl -XGET localhost:9200/_cat/snapshots/esild_backup
  376681 | The average number of logs between 2:00 and 3:00 is 9428. We reached 75572.
  380583 | The average number of logs

---

In [43]:
out_ids = list(range(1000000, 1000000 + len(out_levels)))

In [44]:
out_df = {"id": out_ids, "level": out_levels}

In [45]:
out_df = pd.DataFrame(out_df)

In [46]:
out_df

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [48]:
out_df.to_csv(
    "results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_1-out_ver2.csv", index=False
)